## Seminar 3: Cold start
In this seminar the focus is on cold start (bootstrapping). Cold start is the situation where only a limited amount of training data is avilable that is insufficient to make accurate predictions. In this situation we could start a Machine teaching session with an interface that can allow a user to add labeled items..


We will use the following vocabulary and limit our scope to an ML artifact that does classification from samples; e.g. selects a label for a class, for example cat or dog. We thus define label as the class name. We call the samples we classify training data. One sample has from one to many parameters. To train the ML artifact, samples (consisting of the parameters defining the sample and the correct label) is exposed to the ML artifact when it is in a learning state. The result of the training depends on the quality of this training data for the samples. The ML artifact implements an algorithm (Neural Network, decision tree, etc) that is adapted to the classification problem at hand. When in use for predicting the correct class the ML artifact is exposed to unlabelled samples and infers a prediction of a class label with some accuracy. Cold start/bootstrapping implies that the ML artifact is untrained for the actual context and cannot do any predictions before an initial training has taken place. 



In [51]:
import numpy as np
import pandas as pd
from fastai.imports import *
# from fastai.transforms import *
from fastai.conv_learner import *
# from fastai.model import *
# from fastai.dataset import *
# from fastai.sgdr import *
# from fastai.plots import *
from fastai.structured import *
from fastai.column_data import *
# import torch
# import torch.utils.data
np.set_printoptions(threshold=50, edgeitems=20)
PATH = "data/foo.csv"
df = pd.read_csv(PATH)
#print(df)
cont_parameters =["geoHash","minuteofday"]
cat_parameters = ["detectedActivity","weekday"]

#train_ratio = 0.75
# train_ratio = 0.9
#train_size = int(samp_size * train_ratio); train_size
#val_idx = list(range(train_size, len(df)))
val_idxs = get_cv_idxs(n, val_pct=1.0)
train_size = int(0.8 * len(df))
val_size = len(df) - train_size
print(len(df))
print(val_size+test_size)
#train_dataset, val_dataset = random_split(df, [train_size, val_size])
#print(train_dataset)
#print(val_dataset)

6759
2704


In [104]:
#ds = df["geoHash","minuteofday","detectedActivity","weekday"]
def get_data():
    PATH = "data/foo.csv"
    df = pd.read_csv(PATH)
    ds, y, nas, mapper = proc_df(df, 'journey', do_scale=True)
    n = len(list(open(PATH)))-1
    val_idxs = get_cv_idxs(n)
    return ColumnarModelData.from_data_frame("data",val_idxs, df, y, cat_flds=cat_parameters, bs=128, test_df=None )

In [105]:
data = get_data()
?? data.get_learner()
#a,b,z =next(iter(data.val_dl))
#list(zip(data.classes, y[0]))
#print(a)
#print(b)
#print(z)
#md.get_learner
#1e-3
#m.fit(1e-3, 3)

Signature: data.get_learner(emb_szs, n_cont, emb_drop, out_sz, szs, drops, y_range=None, use_bn=False, **kwargs)
Docstring: <no docstring>
Source:   
    def get_learner(self, emb_szs, n_cont, emb_drop, out_sz, szs, drops,
                    y_range=None, use_bn=False, **kwargs):
        model = MixedInputModel(emb_szs, n_cont, emb_drop, out_sz, szs, drops, y_range, use_bn, self.is_reg, self.is_multi)
        return StructuredLearner(self, StructuredModel(to_gpu(model)), opt_fn=optim.Adam, **kwargs)
File:      /storage/interactive_learning/Seminar_3/fastai/column_data.py
Type:      method


In [106]:
cont_parameters =["geoHash","minuteOfDay"]
cat_parameters = ["detectedActivity","weekday"]
PATH = "data/foo.csv"
df = pd.read_csv(PATH)
dep = "count"
df_full = df[cat_parameters+cont_parameters].copy()
#cat_sz = [(c, len(df[c].cat.categories)+1) for c in cat_parameters]
n = len(df_full); n

6759

In [107]:
for v in cat_parameters: df_full[v] = df_full[v].astype('category').cat.as_ordered()
for v in cont_parameters: df_full[v] = df_full[v].astype('float32')

In [108]:
samp_size = n
df_samp = df_full
cat_sz = [(c, len(df_samp[c].cat.categories)+1) for c in cat_parameters];cat_sz
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz];emb_szs

[(4, 2), (8, 4)]

In [109]:
ds, y, nas, mapper = proc_df(df, 'journey', do_scale=True)
ds.columns
#Fattar inte
yl = np.log(y + 0.01)
max_log_y = np.max(yl)
y_range = (0, max_log_y*1.2)

#m = data.get_learner(emb_szs, len(df.columns)-len(cat_vars),
 #                  0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
#df, y, nas, mapper = proc_df(df_samp, 'count', do_scale=True)

In [110]:
m = data.get_learner(emb_szs, len(df.columns)-len(cat_parameters), 0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)

In [114]:
m.lr_find()
m.sched.plot(1)

RuntimeError: cuda runtime error (59) : device-side assert triggered at /opt/conda/conda-bld/pytorch_1518244421288/work/torch/csrc/generic/serialization.cpp:38

In [113]:
lr = 10e-5
m.fit(lr, 2)

  0%|          | 0/43 [00:00<?, ?it/s]


RuntimeError: cuda runtime error (59) : device-side assert triggered at /opt/conda/conda-bld/pytorch_1518244421288/work/torch/lib/THC/generic/THCTensorCopy.c:20

In [66]:
?? ColumnarModelData

Init signature: ColumnarModelData(path, trn_ds, val_ds, bs, test_ds=None, shuffle=True)
Docstring:      Encapsulates DataLoaders and Datasets for training, validation, test. Base class for fastai *Data classes.
Source:        
class ColumnarModelData(ModelData):
    def __init__(self, path, trn_ds, val_ds, bs, test_ds=None, shuffle=True):
        test_dl = DataLoader(test_ds, bs, shuffle=False, num_workers=1) if test_ds is not None else None
        super().__init__(path, DataLoader(trn_ds, bs, shuffle=shuffle, num_workers=1),
            DataLoader(val_ds, bs*2, shuffle=False, num_workers=1), test_dl)

    @classmethod
    def from_arrays(cls, path, val_idxs, xs, y, is_reg=True, is_multi=False, bs=64, test_xs=None, shuffle=True):
        ((val_xs, trn_xs), (val_y, trn_y)) = split_by_idx(val_idxs, xs, y)
        test_ds = PassthruDataset(*(test_xs.T), [0] * len(test_xs), is_reg=is_reg, is_multi=is_multi) if test_xs is not None else None
        return cls(path, PassthruDataset(*(trn_xs

In [57]:
? proc_df

Signature: proc_df(df, y_fld=None, skip_flds=None, ignore_flds=None, do_scale=False, na_dict=None, preproc_fn=None, max_n_cat=None, subset=None, mapper=None)
Docstring:
proc_df takes a data frame df and splits off the response variable, and
changes the df into an entirely numeric dataframe.

Parameters:
-----------
df: The data frame you wish to process.

y_fld: The name of the response variable

skip_flds: A list of fields that dropped from df.

ignore_flds: A list of fields that are ignored during processing.

do_scale: Standardizes each column in df. Takes Boolean Values(True,False)

na_dict: a dictionary of na columns to add. Na columns are also added if there
    are any missing values.

preproc_fn: A function that gets applied to df.

max_n_cat: The maximum number of categories to break into dummy values, instead
    of integer codes.

subset: Takes a random subset of size subset from df.

mapper: If do_scale is set as True, the mapper variable
    calculates the values used for 

In [24]:
?? get_cv_idxs

Signature: get_cv_idxs(n, cv_idx=0, val_pct=0.2, seed=42)
Docstring:
Get a list of index values for Validation set from a dataset

Arguments:
    n : int, Total number of elements in the data set.
    cv_idx : int, starting index [idx_start = cv_idx*int(val_pct*n)] 
    val_pct : (int, float), validation set percentage 
    seed : seed value for RandomState
    
Returns:
    list of indexes 
Source:   
def get_cv_idxs(n, cv_idx=0, val_pct=0.2, seed=42):
    """ Get a list of index values for Validation set from a dataset
    
    Arguments:
        n : int, Total number of elements in the data set.
        cv_idx : int, starting index [idx_start = cv_idx*int(val_pct*n)] 
        val_pct : (int, float), validation set percentage 
        seed : seed value for RandomState
        
    Returns:
        list of indexes 
    """
    np.random.seed(seed)
    n_val = int(val_pct*n)
    idx_start = cv_idx*n_val
    idxs = np.random.permutation(n)
    return idxs[idx_start:idx_start+n_val]
File

In [37]:
y=df["journey"]
print(y)

0       8121680140
1       8000081216
2       8000081216
3       8121680000
4       8121680000
5       8121680000
6       8121680000
7       8000081216
8       8121680000
9       8121680000
10      8121680140
11      8121680140
12      8000081216
13      9004281216
14      8000081216
15      9004281216
16      8121680000
17      8121680000
18      8000081216
19      8121680000
20      8014081216
21      8121680140
22      8121680000
23      8000081216
24      8121680140
25      8121680000
26      8000081216
27      8000081216
28      8121680140
29      8121680140
           ...    
6729    8121680000
6730    8000081216
6731    8014081216
6732    8121690042
6733    8014081216
6734    8121680000
6735    8121690042
6736    8121680000
6737    8000081216
6738    8000081216
6739    8014081216
6740    8000081216
6741    8121680000
6742    8000081216
6743    8000081216
6744    8121680000
6745    8121690042
6746    8000081216
6747    8121680000
6748    8121680000
6749    8014081216
6750    8000

In [29]:
?? ColumnarModelData

Init signature: ColumnarModelData(path, trn_ds, val_ds, bs, test_ds=None, shuffle=True)
Docstring:      Encapsulates DataLoaders and Datasets for training, validation, test. Base class for fastai *Data classes.
Source:        
class ColumnarModelData(ModelData):
    def __init__(self, path, trn_ds, val_ds, bs, test_ds=None, shuffle=True):
        test_dl = DataLoader(test_ds, bs, shuffle=False, num_workers=1) if test_ds is not None else None
        super().__init__(path, DataLoader(trn_ds, bs, shuffle=shuffle, num_workers=1),
            DataLoader(val_ds, bs*2, shuffle=False, num_workers=1), test_dl)

    @classmethod
    def from_arrays(cls, path, val_idxs, xs, y, is_reg=True, is_multi=False, bs=64, test_xs=None, shuffle=True):
        ((val_xs, trn_xs), (val_y, trn_y)) = split_by_idx(val_idxs, xs, y)
        test_ds = PassthruDataset(*(test_xs.T), [0] * len(test_xs), is_reg=is_reg, is_multi=is_multi) if test_xs is not None else None
        return cls(path, PassthruDataset(*(trn_xs